In [49]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Feature Engineering

## The Data

For this notebook, we will use the seaborn `mpg` data set -- built into the library -- which describes the fuel mileage (measured in miles per gallon, or mpg) of various cars along with characteristics of those cars. Our goal will be to build a model that can predict the fuel mileage of a car based on the characteristics of that car.

**Note:** The feature engineering done in this notebook is not indicative of a feature engineering process you would do when building your own models. The steps taken in this taken are chosen in a specific order and manner to highlight the feature engineering techniques, not optimize the model for performance/quality.

In [50]:
from seaborn import load_dataset
df = load_dataset("mpg")
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


This data has some rows with missing data. We will ignore those rows until later for the sake of this lecture. We can use the Pandas `DataFrame.isna` function to find rows with missing values and drop them.

In [51]:
# the ~ operator is a NOT -- to keep any data which does NOT contain NULL or NaN.  So it will make a copy of the data that does not contain Null
data = df[~df.isna().any(axis=1)].copy()

In [52]:
data
# we can see that some rows of data was removed

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


### Keeping Track of Progress

Because we are going to be building multiple models with different sets of features it is important to have a standard way to track each of the models.  

The following function takes a model prediction function, the name of a model, and the dictionary of models that we have already constructed.  It then evaluates the new model on the data and plots how the new model performs relative to the previous models as well as the $Y$ vs $\hat{Y}$ scatter plot.  

In addition, it updates the dictionary of models to include the new model for future plotting.

*Don't worry about understanding the plotting code and how the dictionary is used.*

In [53]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def evaluate_model(name, model, phi, models=dict()):
    # run the prediction function and compute the RMSE
    Yhat = model.predict(phi(data)).flatten()
    Y = data['mpg'].to_numpy() # the actual mpg values or actual y
    rmse = np.sqrt(mean_squared_error(Y, Yhat))
    print("Root Mean Squared Error:", rmse)
    
    # Save the model and rmse to the collection of models 
    models[name] = dict(model=model, phi=phi, rmse=rmse)
    
    # Generate diagnostic and model comparison plots
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(go.Scatter(x=Yhat, y=Y, mode="markers"), row=1, col=1)
    fig.update_xaxes(title = "Yhat", row=1, col=1)
    fig.update_yaxes(title = "Y", row=1, col=1)
    ymin = np.min(Yhat)
    ymax = np.max(Yhat)
    fig.add_trace(go.Scatter(x=[ymin,ymax], y=[ymin,ymax], name="y=yhat"), row=1, col=1)
    # build a bar graph of the various rmse values for each model name for comparison
    # 
    fig.add_trace(go.Bar(x=list(models.keys()), 
                         y=[models[k]['rmse'] for k in models]), row=1, col=2)    
    fig.update_layout(showlegend=False)
    fig.update_yaxes(title = "RMSE", row=1, col=2)
    fig.show()
    

# this models variable is used later so remember to keep this.
models = {}

## Quantitative Data

This data set has several quantitative features that we can use to build our first model.

Question: which of these columns below are quantitative values?

In [54]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [55]:
# this function focuses on just the quantitative INPUT data columns - notice that it keeps off the mpg which is the output
def basic_design_matrix(df):
    X = df[["cylinders", "displacement", 
          "horsepower", "weight", "acceleration", "model_year"]]
    return X

basic_design_matrix(data)

,cylinders,displacement,horsepower,weight,acceleration,model_year
0,8,307.0,130.0,3504,12.0,70
1,8,350.0,165.0,3693,11.5,70
2,8,318.0,150.0,3436,11.0,70
3,8,304.0,150.0,3433,12.0,70
4,8,302.0,140.0,3449,10.5,70
...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82
394,4,97.0,52.0,2130,24.6,82
395,4,135.0,84.0,2295,11.6,82
396,4,120.0,79.0,2625,18.6,82


In [56]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# the fit API call uses X as first parameter, and Y as the output parameter.
# this model.fit will find the optimal parameters for the model y = theta0 + theta1 * X1 + theta2 * X2, etc.
model.fit(basic_design_matrix(data), data['mpg'])
evaluate_model("basic", model, basic_design_matrix, models)  # this function is defined above and shows the RMSE below.

Root Mean Squared Error: 3.4044340177796406


Interpretting the graphs above

The Y and Yhat would be a perfect straight line and slope would be 1 where y = yHat.  Since they are not on the straight line, the current model isnt that perfect, or a best fit.

The various values of RMSE shows the relative values of RMSE for various models with the same data.  Values closer to zero are better.

We can engineering new features from these existing quantitative features -- with the goal to improve the model.

### Nonlinear transformations

We can compute non-linear transformations of quantitative features such as:
- logarithm
- exponential
- polynomial
- sinusoidal

In [57]:
# We add new columns for each quantitative data column and apply some non-linear function to transform the data


def nonlinear_design_matrix(df):
    X = basic_design_matrix(df)
    # Compute nonlinear transformations for each quantitative feature
    for col in X.columns:
        X[f'{col}^2'] = X[f'{col}'] ** 2
        X[f'{col}^3'] = X[f'{col}'] ** 3
        X[f'log({col})'] = np.log(X[f'{col}'])
        X[f'sin({col})'] = np.sin(X[f'{col}'])
    return X

nonlinearextradata = nonlinear_design_matrix(data)
nonlinearextradata

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,log(weight),sin(weight),acceleration^2,acceleration^3,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year)
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,8.161660,-0.901919,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,8.214194,-0.998328,132.25,1520.875,2.442347,-0.875452,4900,343000,4.248495,0.773891
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,8.142063,-0.784794,121.00,1331.000,2.397895,-0.999990,4900,343000,4.248495,0.773891
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,8.141190,0.689480,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,8.145840,-0.451757,110.25,1157.625,2.351375,-0.879696,4900,343000,4.248495,0.773891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,7.933797,0.262608,243.36,3796.416,2.747271,0.107754,6724,551368,4.406719,0.313229
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,7.663877,0.000181,605.16,14886.936,3.202746,-0.507897,6724,551368,4.406719,0.313229
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,7.738488,0.997785,134.56,1560.896,2.451005,-0.822829,6724,551368,4.406719,0.313229
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,7.872836,-0.980198,345.96,6434.856,2.923162,-0.246974,6724,551368,4.406719,0.313229


In [58]:
model = LinearRegression()
model.fit(nonlinearextradata, data['mpg'])
evaluate_model("nonlinear", model, nonlinear_design_matrix, models)
# is this a better model ?

Root Mean Squared Error: 2.4964145826903583


### Domain Knowledge

As data scientists, you will need domain expertise in the area which you are working in (either you yourself will be a domain expert or you will collaborate with a domain expert).

In the case of the `mpg` dataset, let's say we asked an engine expert and learned the following:

"The displacement of an engine is defined as the product of the volume of each cylinder and number of cylinders.  However, not all cylinders fire at the same time (at least in a functioning engine) so the fuel economy might be more closely related to the volume of any one cylinder." 

We can use this "domain knowledge" to compute a new feature encoding the volume per cylinder by taking the ratio of displacement and cylinders. 

In [59]:
def displacement_design_matrix(df):
    X = nonlinear_design_matrix(df)
    X['displacement/cylinder'] = X['displacement'] / X['cylinders']
    return X

displacement_design_matrix(data)

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,sin(weight),acceleration^2,acceleration^3,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,-0.901919,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.375
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,-0.998328,132.25,1520.875,2.442347,-0.875452,4900,343000,4.248495,0.773891,43.750
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,-0.784794,121.00,1331.000,2.397895,-0.999990,4900,343000,4.248495,0.773891,39.750
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,0.689480,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.000
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,-0.451757,110.25,1157.625,2.351375,-0.879696,4900,343000,4.248495,0.773891,37.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,0.262608,243.36,3796.416,2.747271,0.107754,6724,551368,4.406719,0.313229,35.000
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,0.000181,605.16,14886.936,3.202746,-0.507897,6724,551368,4.406719,0.313229,24.250
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,0.997785,134.56,1560.896,2.451005,-0.822829,6724,551368,4.406719,0.313229,33.750
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,-0.980198,345.96,6434.856,2.923162,-0.246974,6724,551368,4.406719,0.313229,30.000


In [60]:
model = LinearRegression()
model.fit(displacement_design_matrix(data), data['mpg'])
evaluate_model("displacement", model, displacement_design_matrix, models)

Root Mean Squared Error: 2.4900607091337914


In [61]:
# using dataframe percentage to remove outliers
owners = pd.DataFrame({
    'ownerid': [1, 2, 3, 4, 5],
    'name': ['Sam', 'Joe', 'Jill', 'Sue', 'Laura'],
    'age': [2, 12, 9, 9, 99]
})

In [62]:
owners['age'].quantile(0.95)

81.59999999999998

In [63]:
owners['age'].quantile(0.05)

3.4000000000000004

In [64]:
upper = np.array(owners['age'].quantile(0.95) < owners['age'], dtype=bool)
owners[upper]


,ownerid,name,age
4,5,Laura,99


In [65]:
lower = np.array(owners['age'] > owners['age'].quantile(0.05), dtype=bool)


In [66]:
ownerlist = np.logical_and(upper, lower)

In [67]:
ownerlist

array([False, False, False, False,  True])

In [68]:
owners[ownerlist]

,ownerid,name,age
4,5,Laura,99


## Categorical Data

The `origin` column in this data set is categorical (nominal) data taking on a fixed set of possible values.

In [69]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [70]:
px.histogram(data, x='origin')

To use this kind of data in a model, we need to transform into a vector encoding that treats each distinct value as a separate dimension.  This is called One-hot Encoding. 

### One-Hot Encoding

One-Hot encoding, sometimes also called dummy encoding, is a simple mechanism to encode categorical data as real numbers such that the magnitude of each dimension is meaningful.  Suppose a feature can take on $k$ distinct values (e.g., $k=50$ for 50 states in the United Stated).  A new feature (dimension) is created for each distinct value.  For each record, all the new features are set to zero except the one corresponding to the value in the original feature. 

<img src="images/one_hot_state.png" width="600px">

The term one-hot encoding comes from a digital circuit encoding of a categorical state as particular "hot" wire:

<img src="images/one_hot_encoding.png" width="400px">

#### One-Hot Encoding in Pandas

We can construct a one-hot encoding of the origin column using the `pandas.get_dummies` function:

In [71]:
pd.get_dummies(data[['origin']])

,origin_europe,origin_japan,origin_usa
0,False,False,True
1,False,False,True
2,False,False,True
3,False,False,True
4,False,False,True
...,...,...,...
393,False,False,True
394,True,False,False
395,False,False,True
396,False,False,True


Using the `pandas.get_dummies`, we can build a new design matrix which extends our previous features with the additional one-hot encoded columns.

In [72]:
def pandas_ohe_design_matrix(df):
    X = displacement_design_matrix(df)
    return X.join(pd.get_dummies(df[['origin']]))

pandas_ohe_design_matrix(data)

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder,origin_europe,origin_japan,origin_usa
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.375,False,False,True
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,2.442347,-0.875452,4900,343000,4.248495,0.773891,43.750,False,False,True
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,2.397895,-0.999990,4900,343000,4.248495,0.773891,39.750,False,False,True
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.000,False,False,True
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,2.351375,-0.879696,4900,343000,4.248495,0.773891,37.750,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,2.747271,0.107754,6724,551368,4.406719,0.313229,35.000,False,False,True
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,3.202746,-0.507897,6724,551368,4.406719,0.313229,24.250,True,False,False
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,2.451005,-0.822829,6724,551368,4.406719,0.313229,33.750,False,False,True
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,2.923162,-0.246974,6724,551368,4.406719,0.313229,30.000,False,False,True


In [73]:
model = LinearRegression()
model.fit(pandas_ohe_design_matrix(data), data['mpg'])
evaluate_model("ohe", model, pandas_ohe_design_matrix, models)

Root Mean Squared Error: 2.4675755662606536


Unfortunately, there is a problem with our method of one-hot encoding. If we try to use our model to make a prediction on just one data point, the model will fail:

In [74]:
try:
    model.predict(pandas_ohe_design_matrix(data.head(1)))
except Exception as e:
    print(e)

The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- origin_europe
- origin_japan



To see why this fails look at the generated features for the row we are trying to predict:

In [75]:
data.head(1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu


In [76]:
pandas_ohe_design_matrix(data.head(1))

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,acceleration^2,acceleration^3,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder,origin_usa
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,144.0,1728.0,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.375,True


Notice that the columns `origin_japan` and `origin_europe` don't appear. This means there is a mismatch between the number of model parameters and the number of features, which is why the model fails.

There are a couple solutions. We could maintain a list of all the values each categorical variable takes and always make sure to add columns for these values. Alternatively, we could use a library function designed to solve this problem. The second option is much easier.  

#### Scikit-learn One-hot Encoder

The scikit-learn library has a wide range feature transformations and a framework for composing them in reusable (stable) pipelines.  Let's first look at a basic [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) transformation.

In [77]:
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder()

We then fit that instance to some data.  This is where we would determine the specific values that a categorical feature can take:

In [78]:
oh_enc.fit(data[['origin']])

OneHotEncoder()

Once we fit the transformation, we can then use it transform new data:

In [79]:
oh_enc.transform(data[['origin']].head())

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (5, 3)>

In [80]:
oh_enc.transform(data[['origin']].head()).todense()

matrix([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [81]:
data[['origin']].head()

,origin
0,usa
1,usa
2,usa
3,usa
4,usa


We can also inspect the categories of the one-hot encoder:

In [82]:
oh_enc.categories_

[array(['europe', 'japan', 'usa'], dtype=object)]

In [83]:
pd.DataFrame(oh_enc.transform(data[['origin']]).todense(), 
                           columns=oh_enc.categories_,
                           index = data.index)

,europe,japan,usa
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
393,0.0,0.0,1.0
394,1.0,0.0,0.0
395,0.0,0.0,1.0
396,0.0,0.0,1.0


In [84]:
displacement_design_matrix(data)

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,sin(weight),acceleration^2,acceleration^3,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,-0.901919,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.375
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,-0.998328,132.25,1520.875,2.442347,-0.875452,4900,343000,4.248495,0.773891,43.750
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,-0.784794,121.00,1331.000,2.397895,-0.999990,4900,343000,4.248495,0.773891,39.750
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,0.689480,144.00,1728.000,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.000
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,-0.451757,110.25,1157.625,2.351375,-0.879696,4900,343000,4.248495,0.773891,37.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,0.262608,243.36,3796.416,2.747271,0.107754,6724,551368,4.406719,0.313229,35.000
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,0.000181,605.16,14886.936,3.202746,-0.507897,6724,551368,4.406719,0.313229,24.250
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,0.997785,134.56,1560.896,2.451005,-0.822829,6724,551368,4.406719,0.313229,33.750
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,-0.980198,345.96,6434.856,2.923162,-0.246974,6724,551368,4.406719,0.313229,30.000


We can update our design matrix function to use the `OneHotEncoder` we just fitted.

In [85]:
def sklearn_ohe_design_matrix(df):
    X = displacement_design_matrix(df)
    ohe_cols = pd.DataFrame(oh_enc.transform(df[['origin']]).todense(), 
                           columns=oh_enc.categories_,
                           index = df.index)
    return X.join(ohe_cols)
sklearn_ohe_design_matrix(data)

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,log(acceleration),sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder,"(europe,)","(japan,)","(usa,)"
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.375,0.0,0.0,1.0
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,2.442347,-0.875452,4900,343000,4.248495,0.773891,43.750,0.0,0.0,1.0
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,2.397895,-0.999990,4900,343000,4.248495,0.773891,39.750,0.0,0.0,1.0
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,2.484907,-0.536573,4900,343000,4.248495,0.773891,38.000,0.0,0.0,1.0
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,2.351375,-0.879696,4900,343000,4.248495,0.773891,37.750,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,2.747271,0.107754,6724,551368,4.406719,0.313229,35.000,0.0,0.0,1.0
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,3.202746,-0.507897,6724,551368,4.406719,0.313229,24.250,1.0,0.0,0.0
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,2.451005,-0.822829,6724,551368,4.406719,0.313229,33.750,0.0,0.0,1.0
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,2.923162,-0.246974,6724,551368,4.406719,0.313229,30.000,0.0,0.0,1.0


In [86]:
#This one hot encoder approach can be updated using the SciKit Learn package 

## Imputing Missing Values

Let's turn to the issue of handling missing values. To do this, let's bring back the original data.

In [87]:
original_df = df
original_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


Let's see which data is missing:

In [88]:
original_df[original_df.isna().any(axis=1)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
32,25.0,4,98.0,NaN,2046,19.0,71,usa,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,usa,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,europe,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,usa,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,europe,renault 18i
374,23.0,4,151.0,NaN,3035,20.5,82,usa,amc concord dl


There are many ways to deal with missing values.  A common strategy is to substitute the mean.  Because missing values can actually be useful signal, it is often a good idea to include a feature indicating that the value was missing. 

In [89]:
mean_horsepower = data['horsepower'].mean()
mean_horsepower

104.46938775510205

In [90]:
def imputed_design_matrix(df):
    X = df[["cylinders", "displacement", 
          "horsepower", "weight", "acceleration", "model_year"]]
    X["horsepower_missing"] = X["horsepower"].isna()    # this gets converted to 0-1 for prediction
    X = X.fillna(X.mean()) # here the mean is applied for the NA cells.
    return X

imputed_design_matrix(original_df)


,cylinders,displacement,horsepower,weight,acceleration,model_year,horsepower_missing
0,8,307.0,130.0,3504,12.0,70,False
1,8,350.0,165.0,3693,11.5,70,False
2,8,318.0,150.0,3436,11.0,70,False
3,8,304.0,150.0,3433,12.0,70,False
4,8,302.0,140.0,3449,10.5,70,False
...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,False
394,4,97.0,52.0,2130,24.6,82,False
395,4,135.0,84.0,2295,11.6,82,False
396,4,120.0,79.0,2625,18.6,82,False


In [91]:
# we did not update the original_df dataframe -- just in the function
original_df[original_df.isna().any(axis=1)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
32,25.0,4,98.0,NaN,2046,19.0,71,usa,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,usa,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,europe,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,usa,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,europe,renault 18i
374,23.0,4,151.0,NaN,3035,20.5,82,usa,amc concord dl


In [92]:
model = LinearRegression()
model.fit(imputed_design_matrix(original_df), original_df['mpg'])

LinearRegression()

Unfortunately, the feature function we just implemented applies a different transformation depending on what input we provide. Specifically, if the `horsepower` is missing when we go to make a prediction we will substitute it with a different mean than was used when we fit our model.  Furthermore, if we only want predictions on a few records and the `horsepower` is missing from those records then the feature function will be unable to substitute a meaningful value.

For example, if we were to get new data that look like the following:

In [93]:
new_data = original_df[original_df['horsepower'].isna()].head()
new_data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
32,25.0,4,98.0,NaN,2046,19.0,71,usa,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,usa,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,europe,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,usa,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,europe,renault 18i


The feature function is be unable to substitute the mean since none of the rows have a `horsepower` value.

In [94]:
try:
    model.predict(imputed_design_matrix(new_data))
except Exception as e:
    print(e)

Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


We can fix this by computing the mean on the *original* data and using that mean on any new data.

In [95]:
def imputed_design_matrix(df, data_mean=mean_horsepower):
    X = df[["cylinders", "displacement", 
          "horsepower", "weight", "acceleration", "model_year"]]
    X["horsepower_missing"] = X["horsepower"].isna()
    X = X.fillna(data_mean)    # use the mean of the original data
    return X

try:
    model.predict(imputed_design_matrix(new_data))
except Exception as e:
    print(e)

## Quick Reflection

Notice that as we added more features we were able to improve the accuracy of our model.  This is not always a good thing and we will see the problems associated with this in a future lecture.  

It is also worth noting that our design matrix functions each depended on the last. When building your own models, it is recommended you just write one function that takes in the raw data and returns the design matrix. You may write intermediate functions to make your code more modular and readable but there should be one main function that transforms raw input data into a design matrix. An example is given below (this should be helpful for future assignments).

Although we will not cover it in this course, there is another way to deal with feature pipelines using the scikit-learn pipelines module.

In [96]:
def get_design_matrix(df, data_mean=mean_horsepower):
    
    X = df[["cylinders", "displacement", 
          "horsepower", "weight", "acceleration", "model_year"]]
    
    # Compute nonlinear transformations for each quantitative feature (this could be done via a function)
    for col in X.columns:
        X[f'{col}^2'] = X[f'{col}'] ** 2
        X[f'{col}^3'] = X[f'{col}'] ** 3
        X[f'log({col})'] = np.log(X[f'{col}'])
        X[f'sin({col})'] = np.sin(X[f'{col}'])
        
    # Compute the displacement per cylinder (this could be done via a function)
    X['displacement/cylinder'] = X['displacement'] / X['cylinders']
    
    # One-hot encode the categorical columns (this could be done via a function)
    ohe_cols = pd.DataFrame(oh_enc.transform(df[['origin']]).todense(), 
                           columns=oh_enc.categories_,
                           index = df.index)
    X = X.join(ohe_cols)
    
    # Impute missing values (this could be done via a function)
    X["horsepower_missing"] = X["horsepower"].isna()
    X = X.fillna(data_mean)    # use the mean of the original data
    
    return X

get_design_matrix(original_df)

,cylinders,displacement,horsepower,weight,acceleration,model_year,cylinders^2,cylinders^3,log(cylinders),sin(cylinders),...,sin(acceleration),model_year^2,model_year^3,log(model_year),sin(model_year),displacement/cylinder,"(europe,)","(japan,)","(usa,)",horsepower_missing
0,8,307.0,130.0,3504,12.0,70,64,512,2.079442,0.989358,...,-0.536573,4900,343000,4.248495,0.773891,38.375,0.0,0.0,1.0,False
1,8,350.0,165.0,3693,11.5,70,64,512,2.079442,0.989358,...,-0.875452,4900,343000,4.248495,0.773891,43.750,0.0,0.0,1.0,False
2,8,318.0,150.0,3436,11.0,70,64,512,2.079442,0.989358,...,-0.999990,4900,343000,4.248495,0.773891,39.750,0.0,0.0,1.0,False
3,8,304.0,150.0,3433,12.0,70,64,512,2.079442,0.989358,...,-0.536573,4900,343000,4.248495,0.773891,38.000,0.0,0.0,1.0,False
4,8,302.0,140.0,3449,10.5,70,64,512,2.079442,0.989358,...,-0.879696,4900,343000,4.248495,0.773891,37.750,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4,140.0,86.0,2790,15.6,82,16,64,1.386294,-0.756802,...,0.107754,6724,551368,4.406719,0.313229,35.000,0.0,0.0,1.0,False
394,4,97.0,52.0,2130,24.6,82,16,64,1.386294,-0.756802,...,-0.507897,6724,551368,4.406719,0.313229,24.250,1.0,0.0,0.0,False
395,4,135.0,84.0,2295,11.6,82,16,64,1.386294,-0.756802,...,-0.822829,6724,551368,4.406719,0.313229,33.750,0.0,0.0,1.0,False
396,4,120.0,79.0,2625,18.6,82,16,64,1.386294,-0.756802,...,-0.246974,6724,551368,4.406719,0.313229,30.000,0.0,0.0,1.0,False
